In [ ]:
from utils.power_api import fetch_power_api
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# =======================
# Main analysis function
# =======================
def analysis(data):
    # Step 1: fetch NASA POWER data (5 years)
    raw = fetch(data["location"]["lat"], data["location"]["lon"])

    # Step 2: clean dataframe
    df = pandify(raw['properties']['parameter'])

    # Step 3: scope dataframe to requested range
    df_scoped = scope(df, data["date_range"]["start"], data["date_range"]["end"])

    # Step 4: calculate probabilities and colors per activity
    comfort_risk = {}
    for activity in data["activities"]:
        probs = probabilities(df_scoped, activity, data["location"]["name"])
        colors = color_coding(probs)
        comfort_risk[activity] = {
            "probabilities": probs,
            "color_codes": colors,
        }

    return {
        "date_range": data["date_range"],
        "location": data["location"],
        "activities": data["activities"],
        "comfort_risk": comfort_risk,
        "guidance": "Green: Good to go! Blue: Fair. Yellow: Caution. Red: Consider plan B.",
    }

In [ ]:
# =======================
# NASA POWER fetch
# =======================
def fetch(lat, lon):
    return fetch_power_api(
        "daily",
        2020,
        2025,
        lat,
        lon,
        ["T2M", "PRECTOT", "WS10M", "RH2M"],
    )

In [ ]:
# =======================
# Data cleanup helpers
# =======================
def pandify(data: dict) -> pd.DataFrame:
    df = pd.DataFrame(data)
    df = df.replace(-999.0, np.nan).dropna()
    df.index = pd.to_datetime(df.index, format="%Y%m%d")
    return df


def scope(df: pd.DataFrame, start: str, end: str) -> pd.DataFrame:
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")

    start_md = (start_dt.month, start_dt.day)
    end_md = (end_dt.month, end_dt.day)

    mask = (
        (df.index.month > start_md[0])
        | ((df.index.month == start_md[0]) & (df.index.day >= start_md[1]))
    ) & (
        (df.index.month < end_md[0])
        | ((df.index.month == end_md[0]) & (df.index.day <= end_md[1]))
    )

    return df.loc[mask]

: 

In [ ]:

# =======================
# Probabilities + Comfort Score
# =======================
def probabilities(df: pd.DataFrame, activity: str, location: str) -> dict:
    thresholds = {
        "hiking": {"temp": [10,25], "wind speed": [0,20], "humidity": [30,60], "precipitation": [0,2], "comfort score": 4},
        "paragliding": {"temp": [15,28], "wind speed": [10,25], "humidity": [30,65], "precipitation": [0,1], "comfort score": 4},
        "fishing": {"temp": [10,30], "wind speed": [0,25], "humidity": [30,80], "precipitation": [0,3], "comfort score": 4},
    }

    activity_locations = {
        "Chittagong": {"temp": 29, "wind speed": 23, "humidity": 60, "precipitation": 1, "comfort score": 0},
        "Sylhet": {"temp": 29, "wind speed": 23, "humidity": 60, "precipitation": 1, "comfort score": 0}
    }

    # Pick thresholds and actual conditions
    th = thresholds.get(activity, thresholds["hiking"])  
    ac = activity_locations.get(location, activity_locations["Chittagong"])  

    if len(df) == 0:
        return {k: None for k in th}

    # Start fresh comfort score
    score = 0
    max_score = th["comfort score"]

    # Check each parameter
    for param in ["temp", "wind speed", "humidity", "precipitation"]:
        min_val, max_val = th[param]
        current_val = ac[param]

        if min_val <= current_val <= max_val:
            score += 1
        else:
            score -= 1

    # Normalize score (0–100%)
    percentage_score = (score / max_score) * 100

    # Return both raw + normalized score
    ac["comfort score"] = percentage_score
    return ac

In [ ]:
# =======================
# Color coding based on comfort score
# =======================
def color_coding(probs: dict) -> dict:
    """Assign colors based on normalized comfort score (0–100%)."""
    colors = {}
    score = probs.get("comfort score", 0)

    if score >= 100:
        colors["comfort score"] = "green"
    elif score >= 75:
        colors["comfort score"] = "blue"
    elif score >= 50:
        colors["comfort score"] = "yellow"
    else:
        colors["comfort score"] = "red"

    return colors